In [73]:
import mysql.connector
import requests
from json import loads
connection = mysql.connector.connect(host='prd-bcul-mi3',
                                     database='mediainfo',
                                     user='bcul',
                                     password='5#DfbC5u!egsa9Uup')

sql_select_Query = "select * from issues"
cursor = connection.cursor()
cursor.execute(sql_select_Query)
records = cursor.fetchall()
print("Total number of rows in Issues is: ", cursor.rowcount)

Total number of rows in Issues is:  381625


In [107]:
import pickle
with open("bcul_dump.pkl", "w") as f:
    pickle.dump(records, f)

TypeError: write() argument must be str, not bytes

In [90]:
import pandas as pd
columns = ["iss_id", "iss_default_page", "iss_numpages", "iss_date_added", "scanned_rtl", "content_rtl", 
          "pagination", "fs_path", "upload_id", "contentType", "imgRatio", "isDeleted", "bytes", "pixels", 
          "typ", "creator", "assignee", "title", "dsc", "ref", "isHome", "wfStatus"]
drop_cols = ["creator", "assignee", "title", "dsc", "ref", "wfStatus", "scanned_rtl", "content_rtl", 
             "pagination", "bytes", "upload_id", "contentType", "isDeleted", "typ", "isHome"]

query_url = "https://scriptorium.bcu-lausanne.ch/api/item/{}/meta"
df = pd.DataFrame(data=records, columns=columns).drop(columns=drop_cols)
df['metadata_query'] = df.iss_id.apply(lambda x: query_url.format(x))

In [102]:
from dask import bag as db

In [104]:
db.from_sequence(records)

dask.bag<from_sequence, npartitions=101>

In [ ]:
dask.

In [95]:


def get_journal_name(url, iss_id):
    r = requests.get(url)
    if r.status_code != 200:
        print(f"Got status code {r.status_code} for {iss_id}")
        return None
    else:
        c = loads(r.content)
        title = c['title']['values']
        if len(title) == 1:
            title = title[0].replace("<p>", "").replace("</p>", "")
            return title
        else:
            print(f"Got tiles {title} for {iss_id}")
            return None

In [96]:
sample = df.sample(n=10)

In [97]:
sample['journal'] = sample.pr(lambda r: get_journal_name(r.metadata_query, r.iss_id), axis=1)

In [98]:
sample

,iss_id,iss_default_page,iss_numpages,iss_date_added,fs_path,imgRatio,pixels,metadata_query,journal
264432,280309,0,8,2018-04-11 01:31:49,None,NaN,1.770000e+08,https://scriptorium.bcu-lausanne.ch/api/item/2...,Feuille d’avis du district de la Vallée
233946,247718,0,4,2017-10-02 20:10:08,None,NaN,9.235032e+07,https://scriptorium.bcu-lausanne.ch/api/item/2...,Feuille d'avis de Vevey et journal du district
208994,222176,0,16,2017-02-17 23:50:25,None,NaN,3.263955e+08,https://scriptorium.bcu-lausanne.ch/api/item/2...,Feuille d'avis de Vevey et des cercles de La T...
82580,93427,0,4,2015-09-17 02:51:21,mi3_storage/114/1867/12/14/93427,NaN,1.614276e+08,https://scriptorium.bcu-lausanne.ch/api/item/9...,L'Estafette
347157,364693,0,4,2019-02-24 20:36:28,None,NaN,7.482246e+07,https://scriptorium.bcu-lausanne.ch/api/item/3...,"Journal d’Aubonne, feuille d’annonces : organe..."
37432,48076,0,56,2016-01-12 02:21:20,mi3_storage/113/1960/04/21/48076,NaN,2.252399e+09,https://scriptorium.bcu-lausanne.ch/api/item/4...,Feuille d'avis de Lausanne
300928,317937,0,4,2018-10-31 19:09:23,None,NaN,9.161412e+07,https://scriptorium.bcu-lausanne.ch/api/item/3...,"Journal d’Yverdon, Feuille d’Avis d’Yverdon et..."
138727,149591,0,8,2016-01-08 23:05:37,mi3_storage/119/1939/12/19/149591,NaN,3.551484e+08,https://scriptorium.bcu-lausanne.ch/api/item/1...,La Revue
227493,240874,0,4,2017-09-29 01:50:25,None,NaN,8.108208e+07,https://scriptorium.bcu-lausanne.ch/api/item/2...,"Courrier de Vevey, la Tour-de-Peilz et commune..."
307217,324633,0,112,2018-12-11 03:33:11,None,NaN,1.146048e+09,https://scriptorium.bcu-lausanne.ch/api/item/3...,Radio TV - Je vois tout
